In [1]:
# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/'

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)
else:
    print("El código se está ejecutando en un entorno local.")
    path_absolute = 'C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/TFM/WorkSpace/'

path_file = 'df_movies_rating_2.csv'

El código se está ejecutando en un entorno local.


In [2]:
import sys
sys.path.append(path_absolute)


In [3]:
import os
import sys
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

#-------------------------------------------------------------------------------------
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

#-------------------------------------------------------------------------------------
import Utils.utils as util

In [4]:
dataFrame = pd.read_csv(path_absolute+path_file, sep=',')
data = dataFrame.copy()

In [5]:
# Especifica el tamaño de la muestra que deseas obtener
tamano_muestra = 500  # Cambia esto al tamaño de muestra que desees
# Obtiene una muestra aleatoria uniforme del DataFrame
data = data.sample(n=tamano_muestra, random_state=42)  # random_state para reproducibilidad


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 550126 to 190092
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   userId              500 non-null    int64  
 1   movieId             500 non-null    int64  
 2   timestamp_scr       500 non-null    int64  
 3   tag                 500 non-null    object 
 4   tag_encoded         500 non-null    int64  
 5   timestamp_tags      500 non-null    int64  
 6   title               500 non-null    object 
 7   (no genres listed)  500 non-null    int64  
 8   Action              500 non-null    int64  
 9   Adventure           500 non-null    int64  
 10  Animation           500 non-null    int64  
 11  Children            500 non-null    int64  
 12  Comedy              500 non-null    int64  
 13  Crime               500 non-null    int64  
 14  Documentary         500 non-null    int64  
 15  Drama               500 non-null    int64  
 16  

Quito las calumnas categoricas de título y tag ambas estan represnetadas en las columnas númericas de tag_encode e id movie.

In [14]:
# util.contar_ocurrencias(data, '(no genres listed)')
# util.mostrar_filas_por_valor(data, '(no genres listed)', 1)

Total de clases(Puntuaciones) a predecir

In [8]:
data = util.eliminar_columnas(data, ['tag', 'title', '(no genres listed)'])
util.contar_ocurrencias(data, 'rating')

Valor: 5.0, Frecuencia: 112
Valor: 4.0, Frecuencia: 105
Valor: 4.5, Frecuencia: 90
Valor: 3.5, Frecuencia: 86
Valor: 3.0, Frecuencia: 51
Valor: 2.5, Frecuencia: 20
Valor: 2.0, Frecuencia: 18
Valor: 1.0, Frecuencia: 8
Valor: 1.5, Frecuencia: 7
Valor: 0.5, Frecuencia: 3
Total de valores distintos: 10


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 550126 to 190092
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   userId          500 non-null    int64  
 1   movieId         500 non-null    int64  
 2   timestamp_scr   500 non-null    int64  
 3   tag_encoded     500 non-null    int64  
 4   timestamp_tags  500 non-null    int64  
 5   Action          500 non-null    int64  
 6   Adventure       500 non-null    int64  
 7   Animation       500 non-null    int64  
 8   Children        500 non-null    int64  
 9   Comedy          500 non-null    int64  
 10  Crime           500 non-null    int64  
 11  Documentary     500 non-null    int64  
 12  Drama           500 non-null    int64  
 13  Fantasy         500 non-null    int64  
 14  Film-Noir       500 non-null    int64  
 15  Horror          500 non-null    int64  
 16  IMAX            500 non-null    int64  
 17  Musical         500 non-nul

In [9]:
data.head()

,userId,movieId,timestamp_scr,tag_encoded,timestamp_tags,Action,Adventure,Animation,Children,Comedy,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating
550126,64333,42197,1388895420,31405,1388895075,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1.5
722840,15078,3993,1196565802,2699,1296956745,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,5.0
606031,141263,5459,1350545592,9448,1350627511,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1.0
49993,133400,134130,1455373334,4293,1468601068,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,5.0
122599,18057,4226,1166049878,1582,1166050137,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,5.0


MODELO CON KERAS

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Dividir datos en entrenamiento y prueba
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Construir el modelo de recomendación sin escalado
model = Sequential()

# Capas de entrada para datos numéricos
model.add(Dense(64, activation='relu', input_shape=(len(train_data.columns) - 1,)))
model.add(Dense(32, activation='relu'))

# Capa de salida para predicción de puntuación
model.add(Dense(1, activation='linear'))

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Entrenar el modelo
model.fit(train_data.drop('rating', axis=1), train_data['rating'], epochs=10, batch_size=64, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba
loss, mae = model.evaluate(test_data.drop('rating', axis=1), test_data['rating'])
print(f'Mean Absolute Error on Test Data: {mae}')

# Realizar predicciones
predictions = model.predict(test_data.drop('rating', axis=1))


Epoch 1/10
5/5 [==============================] - 3s 250ms/step - loss: 15097519788785664.0000 - mean_absolute_error: 110414296.0000 - val_loss: 32972023529472.0000 - val_mean_absolute_error: 5217486.5000
Epoch 2/10
5/5 [==============================] - 0s 35ms/step - loss: 2309804957630464.0000 - mean_absolute_error: 41150912.0000 - val_loss: 5152445705486336.0000 - val_mean_absolute_error: 71278992.0000
Epoch 3/10
5/5 [==============================] - 0s 24ms/step - loss: 4104552873721856.0000 - mean_absolute_error: 63058092.0000 - val_loss: 1168206140014592.0000 - val_mean_absolute_error: 33783188.0000
Epoch 4/10
5/5 [==============================] - 0s 38ms/step - loss: 405289488613376.0000 - mean_absolute_error: 16531237.0000 - val_loss: 488591822159872.0000 - val_mean_absolute_error: 21908050.0000
Epoch 5/10
5/5 [==============================] - 0s 20ms/step - loss: 1009971022004224.0000 - mean_absolute_error: 31130252.0000 - val_loss: 1178191938977792.0000 - val_mean_absolut

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Concatenate


# Dividir datos en entrenamiento y prueba
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Normalizar datos
# scaler = StandardScaler()
# train_data_scaled = scaler.fit_transform(train_data[['userId', 'movieId', 'timestamp_scr', 'tag_encoded']])
# test_data_scaled = scaler.transform(test_data[['userId', 'movieId', 'timestamp_scr', 'tag_encoded']])

train_data_scaled = train_data[['userId', 'movieId', 'timestamp_scr', 'tag_encoded']]
test_data_scaled = test_data[['userId', 'movieId', 'timestamp_scr', 'tag_encoded']]

# Construir el modelo de recomendación
model = Sequential()

# Capas de entrada para datos numéricos
model.add(Dense(64, activation='relu', input_shape=(train_data_scaled.shape[1],)))
model.add(Dense(32, activation='relu'))

# Capa de salida para predicción de puntuación
model.add(Dense(1, activation='linear'))

#--------------------------------------------------------------------------------------------------------------------------------------------------#

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Entrenar el modelo
model.fit(train_data_scaled, train_data['rating'], epochs=10, batch_size=64, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba
loss, mae = model.evaluate(test_data_scaled, test_data['rating'])
print(f'Mean Absolute Error on Test Data: {mae}')

# Realizar predicciones
predictions = model.predict(test_data_scaled)


Epoch 1/10
5/5 [==============================] - 2s 111ms/step - loss: 427476819902464.0000 - mean_absolute_error: 18187646.0000 - val_loss: 21194115907584.0000 - val_mean_absolute_error: 4578993.0000
Epoch 2/10
5/5 [==============================] - 0s 17ms/step - loss: 142278576308224.0000 - mean_absolute_error: 10989798.0000 - val_loss: 53022264655872.0000 - val_mean_absolute_error: 7242429.0000
Epoch 3/10
5/5 [==============================] - 0s 16ms/step - loss: 70909037641728.0000 - mean_absolute_error: 7315640.0000 - val_loss: 85557308817408.0000 - val_mean_absolute_error: 9200024.0000
Epoch 4/10
5/5 [==============================] - 0s 17ms/step - loss: 39175181041664.0000 - mean_absolute_error: 5405999.0000 - val_loss: 69832032649216.0000 - val_mean_absolute_error: 8311562.5000
Epoch 5/10
5/5 [==============================] - 0s 24ms/step - loss: 29551293890560.0000 - mean_absolute_error: 4569253.0000 - val_loss: 32326612418560.0000 - val_mean_absolute_error: 5655123.0000


MODELO CON TORCH

In [12]:


# Cargar datos (por ejemplo, desde un archivo CSV)
# Suponiendo que tienes un DataFrame llamado 'data' con columnas 'userId', 'movieId' y 'rating'
# ...

# Preprocesamiento de datos
num_movies = data['movieId'].nunique()

# Define el tamaño máximo del índice + 1 (porque los índices comienzan desde 0)
input_dim = max(max(data['userId']), max(data['movieId'])) + 1
num_movies = input_dim

# Crear embeddings para usuarios y películas
user_embedding = nn.Embedding(input_dim, 50)
movie_embedding = nn.Embedding(input_dim, 50)

# Crear conjuntos de entrenamiento y prueba
train, test = train_test_split(data, test_size=0.2, random_state=42)

# Preparar datos para DataLoader
train_users = torch.LongTensor(train['userId'].values)
train_movies = torch.LongTensor(train['movieId'].values)
train_ratings = torch.FloatTensor(train['rating'].values)

test_users = torch.LongTensor(test['userId'].values)
test_movies = torch.LongTensor(test['movieId'].values)
test_ratings = torch.FloatTensor(test['rating'].values)

train_dataset = TensorDataset(train_users, train_movies, train_ratings)
test_dataset = TensorDataset(test_users, test_movies, test_ratings)

# Crear DataLoader
batch_size = 512
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Definir el modelo de recomendación
class Recommender(nn.Module):
    def __init__(self, num_usr, num_movs, embedding_size=50, hidden_size=128):
        super(Recommender, self).__init__()
        self.user_embedding = nn.Embedding(num_usr, embedding_size)
        self.movie_embedding = nn.Embedding(num_movs, embedding_size)
        self.fc1 = nn.Linear(embedding_size * 2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, user, movie):
        user_embed = self.user_embedding(user)
        movie_embed = self.movie_embedding(movie)
        x = torch.cat([user_embed, movie_embed], dim=1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instanciar el modelo, la función de pérdida y el optimizador
model = Recommender(input_dim, num_movies)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenar el modelo
num_epochs = 100

print_interval = int(num_epochs / 10) 

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for user, movie, rating in train_loader:
        optimizer.zero_grad()
        output = model(user, movie).view(-1)
        loss = criterion(output, rating)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * user.size(0)
    train_loss /= len(train_loader.dataset)
    # Imprimir progreso cada 10% de las épocas
    if (epoch + 1) % print_interval == 0 or epoch == num_epochs - 1:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}')
    # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}')

# Evaluar el modelo en el conjunto de prueba
model.eval()
test_loss = 0.0
with torch.no_grad():
    for user, movie, rating in test_loader:
        output = model(user, movie).view(-1)
        print(f'REAL RATINFG: {rating} ------------ PREDICT RATING {output}')
        loss = criterion(output, rating)
        test_loss += loss.item() * user.size(0)
test_loss /= len(test_loader.dataset)
print(f'Test Loss: {test_loss:.4f}')


Epoch [10/100], Loss: 13.8555
Epoch [20/100], Loss: 9.8629
Epoch [30/100], Loss: 6.4519
Epoch [40/100], Loss: 3.7603
Epoch [50/100], Loss: 2.0205
Epoch [60/100], Loss: 1.1443
Epoch [70/100], Loss: 0.7233
Epoch [80/100], Loss: 0.4809
Epoch [90/100], Loss: 0.3314
Epoch [100/100], Loss: 0.2346
REAL RATINFG: tensor([4.0000, 5.0000, 1.5000, 4.5000, 3.0000, 4.5000, 4.0000, 4.5000, 5.0000,
        3.5000, 4.5000, 3.5000, 5.0000, 5.0000, 4.5000, 4.0000, 3.0000, 5.0000,
        4.0000, 4.0000, 2.0000, 4.5000, 3.0000, 4.5000, 5.0000, 4.5000, 4.5000,
        4.5000, 5.0000, 4.0000, 5.0000, 4.0000, 4.5000, 3.0000, 1.0000, 4.5000,
        3.5000, 2.5000, 5.0000, 5.0000, 4.0000, 1.0000, 4.5000, 4.0000, 3.0000,
        3.0000, 1.5000, 4.0000, 4.0000, 4.5000, 4.0000, 4.5000, 4.0000, 4.5000,
        4.0000, 4.5000, 3.5000, 4.0000, 1.0000, 3.5000, 4.0000, 3.5000, 3.0000,
        4.0000, 2.0000, 5.0000, 3.5000, 2.0000, 5.0000, 4.0000, 4.0000, 4.5000,
        2.0000, 3.5000, 4.5000, 3.5000, 3.5000, 3.5000

In [13]:
# # Cargar datos (por ejemplo, desde un archivo CSV)
# # Suponiendo que tienes un DataFrame llamado 'data' con columnas 'userId', 'movieId' y 'rating'
# # ...

# # Preprocesamiento de datos
# # Define el tamaño máximo del índice + 1 (porque los índices comienzan desde 0)
# input_dim = max(max(data['userId']), max(data['movieId'])) + 1

# # Define el tamaño de la capa de embeddings (puedes ajustar este valor según sea necesario)
# embedding_size = 50

# # Crear conjuntos de entrenamiento y prueba
# train, test = train_test_split(data, test_size=0.2, random_state=42)

# # Definir la arquitectura del modelo de recomendación
# user_input = Input(shape=(1,))
# movie_input = Input(shape=(1,))

# user_embedding = Embedding(input_dim=input_dim, output_dim=embedding_size)(user_input)
# movie_embedding = Embedding(input_dim=input_dim, output_dim=embedding_size)(movie_input)

# user_flat = Flatten()(user_embedding)
# movie_flat = Flatten()(movie_embedding)

# concatenated = Concatenate()([user_flat, movie_flat])

# dense_1 = Dense(128, activation='relu')(concatenated)
# output = Dense(1)(dense_1)

# # Crear el modelo
# model = Model(inputs=[user_input, movie_input], outputs=output)

# # Compilar el modelo
# model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01))

# # Entrenar el modelo
# model.fit([train['userId'], train['movieId']], train['rating'], batch_size=512, epochs=100, validation_split=0.2, verbose=0)

# # Evaluar el modelo en el conjunto de prueba
# loss = model.evaluate([test['userId'], test['movieId']], test['rating'])
# print(f'Loss en el conjunto de prueba: {loss}')
